# Computing the Euclidean Distance Matrix with NumPy

In this notebook we implement two functions to compute the Euclidean distance matrix. We use a simple algebra trick that makes possible to write the function in a completely vectorized way in terms of optimized NumPy functions.

In [1]:
import numpy as np

In [2]:
def euclidean_broadcast(x, y):
    """Euclidean square distance matrix.
    
    Inputs:
    x: (N, m) numpy array
    y: (N, m) numpy array
    
    Ouput:
    (N, N) Euclidean square distance matrix:
    r_ij = x_ij^2 - y_ij^2
    """
    diff = x[:, np.newaxis, :] - y[np.newaxis, :, :]

    return (diff * diff).sum(axis=2)

<mark> Question </mark> At this point you are starting to get acquainted with the `numpy.ndarray`s and it's memory managment. Could you analyse the advantage and possible drawbacks of the `euclidean_broadcast` function? Write a positive and a negative point about it.

***

Let's consider now a more sophisticated implementation:

In [3]:
def euclidean_trick(x, y):
    """Euclidean square distance matrix.
    
    Inputs:
    x: (N, m) numpy array
    y: (N, m) numpy array
    
    Ouput:
    (N, N) Euclidean square distance matrix:
    r_ij = x_ij^2 - y_ij^2
    """
    x2 = np.einsum('ij,ij->i', x, x)[:, np.newaxis]
    y2 = np.einsum('ij,ij->i', y, y)[np.newaxis, :]

    xy = np.dot(x, y.T)

    return np.abs(x2 + y2 - 2. * xy)

## The `euclidean_trick` function

Each element of the Euclidean distance matrix is the scalar product of the diference between two rows fo the dataset. `euclidean_trick` takes advantage of this by doing the following
$$\sum_k {(x_{ik}-y_{jk})^2} = (\vec{x}_i - \vec{y}_j)\cdot(\vec{x}_i - \vec{y}_j) = \vec{x}_i\cdot\vec{x}_i + \vec{y}_j\cdot\vec{y}_j - 2\vec{x}_i\cdot\vec{y}_j$$

There are NumPy functions to compute each of these terms:

$\vec{x}_i\cdot\vec{y}_j$ $\rightarrow$ `np.dot(x, y)` : Matrix product of $\{\vec{x}\}$ and $\{\vec{y}\}$

$\vec{x}_i\cdot\vec{x}_i$ $\rightarrow$ `np.einsum('ij,ij->i', x, x)[:, np.newaxis]` : A $(n,1)$ vector of elements $\sum_j x_{ij}x_{ij}$

$\vec{y}_j\cdot\vec{y}_j$ $\rightarrow$ `np.einsum('ij,ij->i', y, y)[np.newaxis, :]` : A $(1,n)$ vector of elements $\sum_j y_{ij}y_{ij}$

To have all the combinations $ij$ of the sum $\vec{x}_i\cdot\vec{x}_i + \vec{y}_j\cdot\vec{y}_j$, we add a new axis to each of the arrays, transpose one them and add them.

Let's see now how the `np.einsum` function works. `einsum` stands for Einstein summation, which is used in tensor algebra to write compact expressions without the sum symbol ($\sum$). Within the Einstein summation notation, whenever there are repeated indexes, there is a sum over them. For instance, the expression
$$x_{ik}y_{kj}$$
is equivalent to
$$\sum_k x_{ik}y_{kj}$$

`np.einsum` uses a generalized form of the Einstein summation by adding the symbol `->` to prevent summing over certain indexes. The specific operation we use here, `np.einsum('ij,ij->i', x, x)`, gives the vector
$$
\begin{bmatrix}
\sum_k x_{1k}x_{1k} \\
\sum_k x_{2k}x_{2k} \\
 ...                \\
\sum_k x_{nk}x_{nk} \\
\end{bmatrix}
$$
Note that the resulting vector is represented here as a column vector just for visualization purposes. It's is `(n,)` NumPy array.

Let's check now step-by-step what the `euclidean_trick` function does:

In [4]:
# Lets generate some random data
nsamples = 10
nfeat = 3

x = 10. * np.random.random([nsamples, nfeat])

In [6]:
x

array([[7.52413725, 0.81874071, 1.08225523],
       [6.82255766, 5.56830485, 5.3885953 ],
       [4.75472335, 9.49855868, 4.50573565],
       [9.42894521, 2.9231334 , 6.75610111],
       [4.65505557, 0.87968187, 8.78403851],
       [7.35359595, 0.7664588 , 9.18966047],
       [0.42007243, 9.49794582, 6.53669695],
       [7.07057793, 8.90325152, 8.6998816 ],
       [4.37518664, 9.73655124, 6.147031  ],
       [7.22248158, 3.82786273, 5.73062056]])

In [11]:
x2 = np.einsum('ij,ij->i', x, x)
x2.shape
print(x2)

[ 58.45425409 106.59027131 133.13166497 143.0946188   99.60271518
 139.11269194 133.11584265 204.94889989 151.72867842  99.6567853 ]


In [8]:
x2

array([ 58.45425409, 106.59027131, 133.13166497, 143.0946188 ,
        99.60271518, 139.11269194, 133.11584265, 204.94889989,
       151.72867842,  99.6567853 ])

In [12]:
x2 = np.einsum('ij,ij->i', x, x)[:, np.newaxis]
x2.shape
print(x)

[[7.52413725 0.81874071 1.08225523]
 [6.82255766 5.56830485 5.3885953 ]
 [4.75472335 9.49855868 4.50573565]
 [9.42894521 2.9231334  6.75610111]
 [4.65505557 0.87968187 8.78403851]
 [7.35359595 0.7664588  9.18966047]
 [0.42007243 9.49794582 6.53669695]
 [7.07057793 8.90325152 8.6998816 ]
 [4.37518664 9.73655124 6.147031  ]
 [7.22248158 3.82786273 5.73062056]]


In [13]:
(x2 + x2.T)#.shape

array([[116.90850818, 165.0445254 , 191.58591907, 201.54887289,
        158.05696928, 197.56694603, 191.57009674, 263.40315398,
        210.18293251, 158.11103939],
       [165.0445254 , 213.18054261, 239.72193628, 249.6848901 ,
        206.19298649, 245.70296325, 239.70611396, 311.5391712 ,
        258.31894973, 206.24705661],
       [191.58591907, 239.72193628, 266.26332995, 276.22628377,
        232.73438016, 272.24435691, 266.24750763, 338.08056486,
        284.8603434 , 232.78845028],
       [201.54887289, 249.6848901 , 276.22628377, 286.1892376 ,
        242.69733398, 282.20731074, 276.21046145, 348.04351869,
        294.82329722, 242.7514041 ],
       [158.05696928, 206.19298649, 232.73438016, 242.69733398,
        199.20543037, 238.71540713, 232.71855784, 304.55161507,
        251.33139361, 199.25950049],
       [197.56694603, 245.70296325, 272.24435691, 282.20731074,
        238.71540713, 278.22538388, 272.22853459, 344.06159183,
        290.84137036, 238.76947724],
       [19

We now use `np.dot` to perform the matrix multiplication of the full dataset by itself. We didn't use it before as alternative to `np.einsum` because it doesn't perform row by row scalar products. Instead `np.dot` expects two arrays with matching shapes $(m,n)$ and $(n,m)$ to perform a matrix multiplication.

We could have used `np.einsum('ik,jk', x, x)` to perform the matrix multiplication, but we chose `np.dot(x, x.T)` instead. This is because `np.dot` is a very sophisticated too, plus it uses OpenMP threads. This results in a very fast execution.

You are wellcome to time them and look at the `top` command to see how `np.dot` uses multiple OpenMP threads.

In [8]:
xy = np.dot(x, x.T)
xy.shape

(10, 10)

Now, considering that the reason we are using `np.einsum` is to get rid of the loops, why didn't we use something like `(x*x).sum(axis=1)`? Let's run the next cell comparing them:

In [20]:
# let's use a larger array for timing the function calls
nsamples = 1000
nfeat = 300

x = 10. * np.random.random([nsamples, nfeat])

# it gives the same result
print(np.abs(np.einsum('ij,ij->i', x, x) - (x*x).sum(axis=1)).max())

# but it's not as fast as `np.einsum`
%timeit -n 10000 -r 5 np.einsum('ij,ij->i', x, x)
%timeit -n 10000 -r 5 (x*x).sum(axis=1)

1.0913936421275139e-11
131 µs ± 1.05 µs per loop (mean ± std. dev. of 5 runs, 10000 loops each)
243 µs ± 71.6 ns per loop (mean ± std. dev. of 5 runs, 10000 loops each)


Doing a reduction with the ufunc `np.add` is also slower than `np.einsum`

In [10]:
%timeit np.add.reduce(x*x, axis=1)

# As homework, check what's the meaning of the previous line!

490 µs ± 170 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Finally, let's time both implementations and check that they give the same result!

In [11]:
nsamples = 2000
nfeat = 50

x = 10. * np.random.random([nsamples, nfeat])

%timeit euclidean_broadcast(x, x)
%timeit euclidean_trick(x, x)

773 ms ± 5.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
36.9 ms ± 539 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
np.abs(euclidean_broadcast(x, x) - euclidean_trick(x, x)).max()

3.296918293926865e-12

<mark> Optional question </mark> Change the implementation of `euclidean_broadcast` function to make faster using `einsum` to do the final sum. How much is the speed up? Compare it with both the original `euclidean_broadcast` and `euclidean_trick`. Check that the result is the same!

In [21]:
# %load solutions/edm-broadcast-einsum.py
idean_broadcast(x, y):
    """Euclidean square distance matrix.
    
    Inputs:
    x: (N,) numpy array
    y: (N,) numpy array
    
    Ouput:
    (N, N) Euclidean square distance matrix:
    r_ij = x_ij^2 - y_ij^2
    """
    diff = x[:, np.newaxis, :] - y[np.newaxis, :, :]

    return np.einsum('ijk,ijk->ij', diff, diff)


# Conclusions

The main points to take from this notebook are:
  * NumPy is all about vectorization. Loops in python must be avoided.
  * Always consider different vectorized implementations and compare them.
  * Even within NumPy, some functions might bring a more significant speedup than others.